<a href="https://colab.research.google.com/github/Space9999/braintumorpredictor/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, glob
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import tensorflow as tf
import random
import pandas as pd
import cv2
sns.set()

In [2]:
ctHealthyPath = glob.glob(os.path.join("/content/drive/MyDrive/Colab Notebooks/Images/Dataset/Brain Tumor CT scan Images", "Healthy", "*.jpg"))
ctTumorPath  = glob.glob(os.path.join("/content/drive/MyDrive/Colab Notebooks/Images/Dataset/Brain Tumor CT scan Images", "Tumor", "*.jpg"))
mriHealthyPath = glob.glob(os.path.join("/content/drive/MyDrive/Colab Notebooks/Images/Dataset/Brain Tumor MRI images", "Healthy", "*.jpg"))
mriTumorPath  = glob.glob(os.path.join("/content/drive/MyDrive/Colab Notebooks/Images/Dataset/Brain Tumor MRI images", "Tumor", "*.jpg"))
data = []

for path in ctHealthyPath:
    data.append((path, 0, "CT"))
for path in ctTumorPath:
    data.append((path, 1, "CT"))

for path in mriHealthyPath:
    data.append((path, 0, "MRI"))
for path in mriTumorPath:
    data.append((path, 1, "MRI"))

print("Number of Data entries:")
print("CT Healthy: ", len(ctHealthyPath))
print("CT Tumor: ", len(ctTumorPath))
print("MRI Healthy: ", len(mriHealthyPath))
print("MRI Tumor: ", len(mriTumorPath))
print("Total images in Data:", len(data))

Number of Data entries:
CT Healthy:  1613
CT Tumor:  2054
MRI Healthy:  1849
MRI Tumor:  2784
Total images in Data: 8300


In [3]:
labels = [i[1] for i in data]
train_paths, val_paths, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)
val_datagen = ImageDataGenerator(rescale=1.0/255)

In [5]:
def generate_images(image_paths, labels, batch_size=32, target_size=(224, 224)):
    # Ensure the generator always yields something, even if there are no images
    if not image_paths:  # If image_paths is empty
        yield np.empty((0, *target_size, 3)), np.empty((0,))  # yield empty arrays
    else:
        while True:
            for i in range(0, len(image_paths), batch_size):
                batch_paths = image_paths[i:i + batch_size]
                batch_labels = labels[i:i + batch_size]

                images = []
                for img_data in batch_paths:
                    img_path = img_data[0]
                    img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
                    img = tf.keras.preprocessing.image.img_to_array(img)
                    images.append(img)

                images = np.array(images)
                yield images, np.array(batch_labels)
train_generator = generate_images(train_paths, train_labels, batch_size=32)
val_generator = generate_images(val_paths, val_labels, batch_size=32)

In [6]:
from tensorflow.keras.applications import EfficientNetV2B0
base_model = EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

#  Fine-Tuning
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # one output (tumor / healthy)

# bulid the model
model = Model(inputs=base_model.input, outputs=predictions)

# while training , trainable = false
for layer in base_model.layers:
    layer.trainable = False


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=len(train_paths) // 32,
    validation_data=val_generator,
    validation_steps=len(val_paths) // 32,
    epochs=7
)

Epoch 1/7
207/207 ━━━━━━━━━━━━━━━━━━━━ 597s 3s/step - accuracy: 0.9084 - loss: 0.2096 - val_accuracy: 0.9724 - val_loss: 0.0856
Epoch 2/7
207/207 ━━━━━━━━━━━━━━━━━━━━ 551s 3s/step - accuracy: 0.9795 - loss: 0.0616 - val_accuracy: 0.9822 - val_loss: 0.0591
Epoch 3/7
207/207 ━━━━━━━━━━━━━━━━━━━━ 548s 3s/step - accuracy: 0.9848 - loss: 0.0405 - val_accuracy: 0.9840 - val_loss: 0.0545
Epoch 4/7
207/207 ━━━━━━━━━━━━━━━━━━━━ 548s 3s/step - accuracy: 0.9890 - loss: 0.0305 - val_accuracy: 0.9834 - val_loss: 0.0471
Epoch 5/7
207/207 ━━━━━━━━━━━━━━━━━━━━ 547s 3s/step - accuracy: 0.9908 - loss: 0.0277 - val_accuracy: 0.9760 - val_loss: 0.0664
Epoch 6/7
207/207 ━━━━━━━━━━━━━━━━━━━━ 545s 3s/step - accuracy: 0.9887 - loss: 0.0267 - val_accuracy: 0.9883 - val_loss: 0.0349
Epoch 7/7
207/207 ━━━━━━━━━━━━━━━━━━━━ 549s 3s/step - accuracy: 0.9925 - loss: 0.0190 - val_accuracy: 0.9859 - val_loss: 0.0518


In [7]:
val_loss, val_accuracy = model.evaluate(val_generator, steps=len(val_paths) // 32)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

51/51 ━━━━━━━━━━━━━━━━━━━━ 107s 2s/step - accuracy: 0.9856 - loss: 0.0603
Validation Loss: 0.05177828297019005
Validation Accuracy: 0.985872209072113


In [8]:
model.save("model2.keras")

In [9]:
ctHealthyPathVal = glob.glob(os.path.join("/content/drive/MyDrive/Colab Notebooks/Images/Dataset", "CT_HealthyTest", "*.jpg"))
ctTumorPathVal  = glob.glob(os.path.join("/content/drive/MyDrive/Colab Notebooks/Images/Dataset", "CT_TumorTest", "*.jpg"))
mriHealthyPathVal = glob.glob(os.path.join("/content/drive/MyDrive/Colab Notebooks/Images/Dataset", "MRI_HealthyTest", "*.jpg"))
mriTumorPathVal  = glob.glob(os.path.join("/content/drive/MyDrive/Colab Notebooks/Images/Dataset", "MRI_TumorTest", "*.jpg"))

testData = []

for path in ctHealthyPathVal:
    testData.append((path, 0, "CT"))
for path in ctTumorPathVal:
    testData.append((path, 1, "CT"))

for path in mriHealthyPathVal:
    testData.append((path, 0, "MRI"))
for path in mriTumorPathVal:
    testData.append((path, 1, "MRI"))

print("Number of Test Data entries:")
print("CT Healthy: ", len(ctHealthyPathVal))
print("CT Tumor: ", len(ctTumorPathVal))
print("MRI Healthy: ", len(mriHealthyPathVal))
print("MRI Tumor: ", len(mriTumorPathVal))
print("Total images in Test Data:", len(testData))

Number of Test Data entries:
CT Healthy:  103
CT Tumor:  103
MRI Healthy:  148
MRI Tumor:  200
Total images in Test Data: 554


In [10]:
val_datagen_2 = ImageDataGenerator(rescale=1.0/255)

In [11]:
def generate_images(image_paths, labels, batch_size=32, target_size=(224, 224)):
    # Ensure the generator always yields something, even if there are no images
    if not image_paths:  # If image_paths is empty
        yield np.empty((0, *target_size, 3)), np.empty((0,))  # yield empty arrays
    else:
        while True:
            for i in range(0, len(image_paths), batch_size):
                batch_paths = image_paths[i:i + batch_size]
                batch_labels = labels[i:i + batch_size]

                images = []
                for img_data in batch_paths:
                    img_path = img_data[0]
                    img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
                    img = tf.keras.preprocessing.image.img_to_array(img)
                    images.append(img)

                images = np.array(images)
                yield images, np.array(batch_labels)
testLabels = [j[1] for j in testData]
val_generator = generate_images(testData, testLabels, batch_size=32)

In [12]:
val_loss, val_accuracy = model.evaluate(val_generator, steps=len(testData) // 4)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

138/138 ━━━━━━━━━━━━━━━━━━━━ 280s 2s/step - accuracy: 0.9198 - loss: 0.3150
Validation Loss: 0.25208625197410583
Validation Accuracy: 0.9343031644821167
